In [ ]:
!nvidia-smi

Sat Jun  1 03:22:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   48C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

transformers 和 Sentence-transformers （使用 huggingface 模型）

In [2]:
!pip install jieba
!pip install sentence_transformers
!pip install ckip_transformers
!pip install bertopic
!pip install scikit-learn-extra

!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

! pip install numpy
! pip install pyvis
! pip install networkx
! pip install IPython
! pip install pyLDAvis
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## 連結雲端、設定環境

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/NSYSU/Social-Media-Analysis/Final') #切換該目錄
os.listdir() #確認目錄內容

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Final.ipynb', 'net_func.py', '__pycache__']

In [5]:
# 環境變數設定
%env PYTHONHASHSEED=8787

env: PYTHONHASHSEED=8787


## 載入套件

In [112]:
import time
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import re
import ast
import json
import seaborn as sns
import torch
import pyvis
import IPython
import random
import networkx as nx
# import names
from itertools import combinations


from collections import defaultdict
import multiprocessing

import pyLDAvis
import pyLDAvis.gensim_models

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec, KeyedVectors, LdaModel, CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.matutils import corpus2csc, corpus2dense, Sparse2Corpus

import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sentence_transformers import SentenceTransformer, models, util
from transformers import BertTokenizerFast, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker


from functools import reduce
from collections import Counter
from pprint import pprint

import nltk
import net_func
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN
from getpass import getpass

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 設定中文字型

In [8]:
# 設定字體
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']
plt.rcParams['font.size'] = '16'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 第一題

In [10]:
# 建立 network
# cdn_resources='in_line' 需要記得加，瀏覽器才可以打開
netWork = pyvis.network.Network(notebook=True, cdn_resources='in_line')

# 一次加入多個節點
netWork.add_nodes(
    nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'], # 設定專屬id
    value = [2,2,2,2,2,2,2,2,2,2], # node大小
    label = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
)

# 加入edge
netWork.add_edges(
    edges=[('A','B',1), ('A','C',1), ('A','D',1), ('B','D',1),
        ('C','D',1), ('E','D',1), ('F','D',1), ('G','F',1),
        ('F','H',1), ('G','I',1), ('F','I',1), ('I','J',1)] #（節點1, 節點2, width）

)

netWork.show("./netWork.html")
IPython.display.HTML('netWork.html')

./netWork.html


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
graph = nx.Graph()

edges = [('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'D'),
      ('C', 'D'), ('E', 'D'), ('F', 'D'), ('G', 'F'),
      ('F', 'H'), ('G', 'I'), ('F', 'I'), ('I', 'J')]

graph.add_edges_from(edges)

# 密度
density = nx.density(graph)
print(f"Density: {density}")

# 平均距離
average_distance = nx.average_shortest_path_length(graph)
print(f"Average Distance: {average_distance}")

# 直徑
diameter = nx.diameter(graph)
print(f"Diameter: {diameter}")

Density: 0.26666666666666666
Average Distance: 2.2222222222222223
Diameter: 4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 第二題

In [32]:
original = pd.read_csv("raw_data/student_2_5_8.csv")  # 匯資料
original.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,artTitle,artContent
0,郭子乾趕高鐵巧遇王世堅 網譏「想看韓院長表示....」,郭子乾（左）和王世堅昨在高鐵站巧遇，開心拍下同框合影。（翻攝自郭子乾臉書）\n〔記者林南谷／...
1,國民黨智庫籲新政府解除不合時宜兩岸禁令 恢復正常往來,2024/05/14 15:49\n\n\n 國民黨智庫表示，國民黨將在智庫國安組召集人陳永...
2,漲電價要立法院同意！藍白再聯手表決通過「電業法」逕付二讀,2024/05/03 11:54\n\n\n 立法院議場上演藍綠表決攻防，國民黨團高喊凍漲電...


In [33]:
# 移除網址格式
# 只留下中文字
metadata = original.copy()
metadata["artContent"] = metadata["artContent"].str.replace("(http|https)://.*", "", regex=True)
metadata["artTitle"] = metadata["artTitle"].str.replace("(http|https)://.*", "", regex=True)
metadata["artContent"] = metadata["artContent"].str.replace("[^\u4e00-\u9fa5]+", "", regex=True)
metadata["artTitle"] = metadata["artTitle"].str.replace("[^\u4e00-\u9fa5]+", "", regex=True)
metadata['content'] = metadata['artContent']

metadata.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,artTitle,artContent,content
0,郭子乾趕高鐵巧遇王世堅網譏想看韓院長表示,郭子乾左和王世堅昨在高鐵站巧遇開心拍下同框合影翻攝自郭子乾臉書記者林南谷台北報導藝人郭子乾月...,郭子乾左和王世堅昨在高鐵站巧遇開心拍下同框合影翻攝自郭子乾臉書記者林南谷台北報導藝人郭子乾月...
1,國民黨智庫籲新政府解除不合時宜兩岸禁令恢復正常往來,國民黨智庫表示國民黨將在智庫國安組召集人陳永康見圖的帶領下於立法院下會期推動國家安全戰略法立...,國民黨智庫表示國民黨將在智庫國安組召集人陳永康見圖的帶領下於立法院下會期推動國家安全戰略法立...
2,漲電價要立法院同意藍白再聯手表決通過電業法逕付二讀,立法院議場上演藍綠表決攻防國民黨團高喊凍漲電價苦民所苦記者田裕華攝記者林欣漢陳政宇李文馨台北...,立法院議場上演藍綠表決攻防國民黨團高喊凍漲電價苦民所苦記者田裕華攝記者林欣漢陳政宇李文馨台北...


In [36]:
# 設定繁體中文詞庫
jieba.set_dictionary("./dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stop_words.txt') #jieba.analyse.extract_tags才會作用
with open("./dict/stop_words.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]
    stopwords_manual = ["表示"]
    stopWords.extend(stopwords_manual)

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

metadata["words"] = metadata["content"].apply(getToken)
metadata.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from /content/drive/MyDrive/Colab Notebooks/NSYSU/Social-Media-Analysis/Final/dict/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/drive/MyDrive/Colab Notebooks/NSYSU/Social-Media-Analysis/Final/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u2001e9ddad76f2a465701723173e06e1.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.u2001e9ddad76f2a465701723173e06e1.cache
Loading model cost 1.251 seconds.
DEBUG:jieba:Loading model cost 1.251 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,artTitle,artContent,content,words
0,郭子乾趕高鐵巧遇王世堅網譏想看韓院長表示,郭子乾左和王世堅昨在高鐵站巧遇開心拍下同框合影翻攝自郭子乾臉書記者林南谷台北報導藝人郭子乾月...,郭子乾左和王世堅昨在高鐵站巧遇開心拍下同框合影翻攝自郭子乾臉書記者林南谷台北報導藝人郭子乾月...,"[郭子, 王世堅, 昨在, 高鐵, 巧遇, 開心, 下同, 合影, 翻攝, 郭子, 乾臉, ..."
1,國民黨智庫籲新政府解除不合時宜兩岸禁令恢復正常往來,國民黨智庫表示國民黨將在智庫國安組召集人陳永康見圖的帶領下於立法院下會期推動國家安全戰略法立...,國民黨智庫表示國民黨將在智庫國安組召集人陳永康見圖的帶領下於立法院下會期推動國家安全戰略法立...,"[國民黨, 智庫, 國民黨, 將在, 智庫, 國安, 召集人, 永康, 見圖, 帶領, 立法..."
2,漲電價要立法院同意藍白再聯手表決通過電業法逕付二讀,立法院議場上演藍綠表決攻防國民黨團高喊凍漲電價苦民所苦記者田裕華攝記者林欣漢陳政宇李文馨台北...,立法院議場上演藍綠表決攻防國民黨團高喊凍漲電價苦民所苦記者田裕華攝記者林欣漢陳政宇李文馨台北...,"[立法院, 議場, 上演, 藍綠, 表決, 攻防, 國民黨, 高喊, 凍漲, 電價, 苦民所..."
3,徐巧芯丈夫被控過去是挺朱網軍朱立倫外界不應牽拖,國民黨智庫日召開新政府新期待別讓人民受傷害記者會國民黨主席朱立倫出席致詞並於會後受訪記者塗建...,國民黨智庫日召開新政府新期待別讓人民受傷害記者會國民黨主席朱立倫出席致詞並於會後受訪記者塗建...,"[國民黨, 智庫, 召開, 新政府, 期待, 受傷害, 記者會, 國民黨, 主席, 朱立倫,..."
4,姓名條例修正草案三讀通過原住民族可於身分證單列族名,立法院長韓國瑜敲下議事槌三讀通過姓名條例部分條文修正草案記者劉信德攝記者林欣漢台北報導立法院...,立法院長韓國瑜敲下議事槌三讀通過姓名條例部分條文修正草案記者劉信德攝記者林欣漢台北報導立法院...,"[立法, 院長, 韓國, 敲下, 議事, 三讀通過, 姓名, 條例, 部分, 條文, 修正,..."


In [37]:
docs = metadata['words'].to_list()
docs[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['郭子',
 '王世堅',
 '昨在',
 '高鐵',
 '巧遇',
 '開心',
 '下同',
 '合影',
 '翻攝',
 '郭子',
 '乾臉',
 '書記',
 '林南谷',
 '台北',
 '報導',
 '藝人',
 '郭子',
 '化身',
 '立法',
 '院長',
 '韓國',
 '前往',
 '立院',
 '拍片',
 '期間',
 '還跑',
 '民進黨',
 '立委',
 '王世堅',
 '辦公室',
 '拜會',
 '事後',
 '王世堅',
 '受訪',
 '當時',
 '嚇到並',
 '稱讚',
 '郭子',
 '模仿',
 '超像',
 '只能',
 '說分',
 '以上',
 '沒想到',
 '郭子',
 '昨在',
 '高鐵',
 '趕路',
 '行程',
 '巧遇',
 '王世堅',
 '兩人',
 '下同',
 '合影',
 '事後',
 '郭子',
 '臉書',
 '照說',
 '節目',
 '模仿',
 '合照',
 '機會',
 '用哥',
 '本尊',
 '合照',
 '直呼',
 '真是',
 '太巧',
 '今年',
 '郭子',
 '前往',
 '立院',
 '拍片',
 '王世堅',
 '模仿',
 '韓國',
 '活靈活現',
 '仔細',
 '研究',
 '韓國',
 '背景',
 '說完',
 '全不',
 '知道',
 '郭子',
 '乾會',
 '立法院',
 '當時',
 '郭子',
 '乾進',
 '來時',
 '還以',
 '真的',
 '韓國',
 '爆笑',
 '郭子',
 '昨曬出',
 '王世堅',
 '同框',
 '意外',
 '釣出',
 '粉絲',
 '網友',
 '留言',
 '很想',
 '看韓',
 '院長']

In [38]:
dictionary = Dictionary(docs)
# Filter out words that occur less than 5 documents, or more than 99% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.99)
print(dictionary)

Dictionary<3870 unique tokens: ['事後', '今年', '以上', '兩人', '前往']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
for idx, (k, v) in enumerate(dictionary.token2id.items()):
    print(f"{k}: {v}")
    if idx > 10:
        break

事後: 0
今年: 1
以上: 2
兩人: 3
前往: 4
受訪: 5
只能: 6
台北: 7
合影: 8
合照: 9
報導: 10
意外: 11


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Bag-of-words representation of the documents.
# 用 gensim ldamodel input 需要將文章轉換成 bag of words
corpus = [dictionary.doc2bow(doc) for doc in docs]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 訓練LDA模型

In [41]:
ldamodel = LdaModel(
    corpus=corpus,
    id2word=dictionary, # 字典
    num_topics=4, # 生成幾個主題數
    random_state=2024, # 亂數
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
ldamodel.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.008*"台灣" + 0.007*"季後賽" + 0.007*"報導" + 0.005*"記者" + 0.004*"籃板" + 0.004*"綜合" + 0.004*"指出" + 0.004*"美國" + 0.004*"立法院" + 0.004*"今天"'),
 (1,
  '0.009*"立委" + 0.009*"記者" + 0.009*"花蓮" + 0.008*"地震" + 0.007*"台灣" + 0.006*"國民黨" + 0.006*"民進黨" + 0.006*"報導" + 0.005*"立法院" + 0.004*"民眾"'),
 (2,
  '0.010*"花蓮" + 0.010*"地震" + 0.009*"記者" + 0.007*"報導" + 0.006*"提供" + 0.005*"指出" + 0.004*"晶片" + 0.004*"民眾" + 0.004*"台灣" + 0.004*"目前"'),
 (3,
  '0.011*"民進黨" + 0.010*"立委" + 0.008*"國民黨" + 0.008*"立法院" + 0.008*"報導" + 0.007*"地震" + 0.007*"記者" + 0.007*"台灣" + 0.006*"花蓮" + 0.006*"國會"')]

In [61]:
# 列出每個主題的前10個詞
num_words = 10
topics = ldamodel.show_topics(num_topics=4, num_words=num_words, formatted=False)

# 輸出每個主題的前10個詞
for i, topic in enumerate(topics):
    print(f"Topic {i + 1}:")
    print([word for word, _ in topic[1]])

Topic 1:
['台灣', '季後賽', '報導', '記者', '籃板', '綜合', '指出', '美國', '立法院', '今天']
Topic 2:
['立委', '記者', '花蓮', '地震', '台灣', '國民黨', '民進黨', '報導', '立法院', '民眾']
Topic 3:
['花蓮', '地震', '記者', '報導', '提供', '指出', '晶片', '民眾', '台灣', '目前']
Topic 4:
['民進黨', '立委', '國民黨', '立法院', '報導', '地震', '記者', '台灣', '花蓮', '國會']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 看LDA指標

In [43]:
# perplexity
perplexity = ldamodel.log_perplexity(corpus)
perplexity

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-7.657131884091611

In [44]:
np.exp2(-perplexity)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


201.84889574925236

In [45]:
# npmi
NPMI_model_lda = CoherenceModel(model=ldamodel, texts=docs, coherence='c_npmi')
NPMI_lda = NPMI_model_lda.get_coherence()
print('這個主題的 PMI score: ', NPMI_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


這個主題的 PMI score:  -0.04191560429777486


In [46]:
NPMI_model_lda.get_coherence_per_topic()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[-0.08142824187007582,
 0.009290106960209635,
 -0.07109115905900545,
 -0.024433123222227814]

### 存成LDAvis

In [47]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
p

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.017904  0.018957       1        1  29.493973
2     -0.005932 -0.019988       2        1  26.313846
0      0.044015  0.003764       3        1  25.466637
1     -0.020179 -0.002733       4        1  18.725544, topic_info=     Term        Freq       Total Category  logprob  loglift
212    花蓮  702.000000  702.000000  Default  30.0000  30.0000
25     立委  674.000000  674.000000  Default  29.0000  29.0000
17    民進黨  633.000000  633.000000  Default  28.0000  28.0000
1040   地震  746.000000  746.000000  Default  27.0000  27.0000
3268  季後賽  381.000000  381.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
529    國會   61.072319  347.561195   Topic4  -5.8336  -0.0636
807    提供   64.604450  402.514207   Topic4  -5.7774  -0.1542
252    今天   56.496563  394.657603   Topic4  -5.9115  -0.2686
148    資料   51.695813  307.758445   Topic4  -6.0003  -0.1087
2899   晶片   52.330443  346.167774   Topic4  -5.9881  -0.2141

[325 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2269      1  0.659963   一併
2269      2  0.146658   一併
2269      3  0.146658   一併
2269      4  0.073329   一併
3440      1  0.193093   三分
...     ...       ...  ...
3282      4  0.045622   鵜鶘
313       1  0.498582   黃國
313       2  0.265910   黃國
313       3  0.091407   黃國
313       4  0.141265   黃國

[916 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])

In [70]:
pyLDAvis.save_html(p, "ldamodel_final.html")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 第三題

輸入 HuggingFace Token
- get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [63]:
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


··········


In [64]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

print(quant_config.is_quantizable())
print(quant_config.quantization_method())

True
fp4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
!pip install -q diffusers transformers accelerate peft

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 28.7 MB/s eta 0:00:00


In [114]:
model_id = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
# device_map="auto" 使用 GPU 加速
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [115]:
original_3 = pd.read_csv("raw_data/emotion_samples.csv")  # 匯資料
original_3.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,label
0,I cant sleep im too worried about ashton and i...,negative
1,Having to be at school at eight tomorrow is go...,negative
2,Three hours of sleep would be a lot worse if i...,negative


### A

In [116]:
fewshot_template = """<start_of_turn>user
you are a genius, you can classify any reviews.

以下為一些例子：
example 1:
review: "{example1_review}"
Label: {example1_label}

example 2:
review: "{example2_review}"
Label: {example2_label}

example 3:
review: "{example3_review}"
Label: {example3_label}

現在請回答以下問題
問題：{review}<end_of_turn>
<start_of_turn>model
答案："""

# 選取三個範例
examples = original_3.sample(3)

# few-shot prompt
fewshot_prompt_text = fewshot_template.format(
    example1_review=examples.iloc[0]['text'],
    example1_label=examples.iloc[0]['label'],
    example2_review=examples.iloc[1]['text'],
    example2_label=examples.iloc[1]['label'],
    example3_review=examples.iloc[2]['text'],
    example3_label=examples.iloc[2]['label'],
    review="{review}"
)

# 建立 PromptTemplate
fewshot_prompt = PromptTemplate.from_template(fewshot_prompt_text)


fewshot_chain = (
    { "review": RunnablePassthrough()}
    | fewshot_prompt
    | llm
)

# 自定義self-consistency 方法
def classify_review_with_consistency(review, chain, n=5):
    results = []
    for _ in range(n):
        result = chain.invoke({"review": review})
        results.append(result.strip())
    most_common_result = Counter(results).most_common(1)[0][0]
    return most_common_result

test_review = original_3.iloc[0]['text']
fewshot_chain.invoke({"Review": test_review})


# 執行 self-consistency
final_label = classify_review_with_consistency(test_review, fewshot_chain)

print(f"Review: {test_review}")
print(f"Predicted Label: {final_label}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Review: I cant sleep im too worried about ashton and i have school tomorrow but fuck school i care more about the person who really saved me
Predicted Label: <start_of_turn>user
you are a genius, you can classify any reviews.

以下為一些例子：
example 1:
review: "I remember in 6th grade when I was in love with Green Day and my mama bought me like all their albums lol she was the real MVP"
Label: positive

example 2:
review: "In Arusha for tomorrow's annual Toyota cocktail and dinner at the HRC. Looking forward to seeing old friends and Toyota customers."
Label: positive

example 3:
review: "Tomorrow's plan: Climbing with Liam. Homemade mini pizzas for lunch. Hangs with Janet &amp; Dani. Dinner &amp; movie with Liam. Then The Following."
Label: neutral

現在請回答以下問題
問題：{'review': 'I cant sleep im too worried about ashton and i have school tomorrow but fuck school i care more about the person who really saved me'}<end_of_turn>
<start_of_turn>model
答案：

Review: "I cant sleep im worried about ashton 

### B

In [ ]:
# 應用到整個資料集
true_labels = original_3['label'].tolist()
predicted_labels = []

for review in original_3['text']:
    predicted_label = classify_review_with_consistency(review, fewshot_chain)
    predicted_labels.append(predicted_label)

# 計算評估指標
accuracy = accuracy_score(true_labels, predicted_labels)
macro_precision = precision_score(true_labels, predicted_labels, average='macro')
macro_recall = recall_score(true_labels, predicted_labels, average='macro')
macro_f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Macro Precision: {macro_precision}")
print(f"Macro Recall: {macro_recall}")
print(f"Macro F1 Score: {macro_f1}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
